In [1]:
from tdmclient import ClientAsync, aw
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

Node a4a5db65-ab23-417b-975e-12649f3406c8

In [2]:
#librairy for kalman
#!pip install filterpy 
#!pip install tqdm
#!pip install pyvisgraph
#!pip install opencv-python
#!pip install opencv-contrib-python --user

In [3]:
import numpy as np
import cv2
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise
import math

In [4]:
global index,speed_left,speed_right,direction,state,pos_cam, new_image
timer_cam=1
dt=0.1
state=0
new_image=0
direction=0
robot_state=np.array([0,0,0,0],'double') # [x,y,vx,vy]
pos_cam=np.array([0,0],'double')
Ts = 0.1
qp = 0.04 # variance on position state
rp = 0.25 # variance on position measurement
q_nu= 0.04 #variance on speed state
r_nu=0.25 #variance on speed measurement
index=0

In [5]:
class Map:
    # compute_globalmap(self,image,robot_size,start_id=0,goal_id=1):
    #  Generate globalmap with obstacle inflation: 0:free 1:occupied 2:start 3:goal
    #  return self.globalmap
    # compute_pixel_to_distance(self):
    #  compute real world distance between two neighborhood pixels;unit:mm
    #  **only have the value when goal is detected successfully
    #  return self.pixel_to_distance
    # compute_start(self,image, start_id=0):
    #  Detect start by aruco marker
    #  Return start position in (x,y) coordinate; if not detected return None
    #  return self.start
    # compute_goal(self,image, goal_id=1):
    #  Detect goal by aruco marker
    #  Return goal position in (x,y) coordinate; if not detected return None
    #  return self.goal
    # get_pixel_to_distance(self):
    #  return self.pixel_to_distance
    # get_globalmap(self):
    #  return self.globalmap
    # get_start(self):
    #  return self.start
    # get_goal(self):
    #  return self.goal

    def compute_globalmap(self, image, robot_size=0, start_id=0, goal_id=1):
        # Generate globalmap with obstacle inflation: 0:free 1:occupied 2:start 3:goal
        # height, width = image.shape[:2]

        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # Simple threshold method
        # ret,thresh = cv2.threshold(gray,50,255,cv2.THRESH_BINARY)


        # Otsu's thresholding after Gaussian filtering
        blur = cv2.GaussianBlur(gray, (5, 5), 0)
        ret, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        # cv2.imshow("thresh", thresh)

        # Generate Global map
        # from 0,255 convert to 0(free),1(occupied);change to (x,y) coordinate
        temp_globalmap = np.transpose(np.array(thresh < 255, dtype=int))

        # Detect start and goal by aruco marker
        # Return start and goal position in (x,y) coordinate; if not detected return None
        # start_id: start aruco marker id
        # goal_id=: goal aruco marker id

        # Define aruco marker detector
        arucoDict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_50)
        arucoParams = cv2.aruco.DetectorParameters_create()

        # Detect aruco marker
        (corners, ids, rejected) = cv2.aruco.detectMarkers(image, arucoDict, parameters=arucoParams)
        # print(corners, ids, rejected)
        start = None
        goal = None
        s = 10
        if ids is not None:
            for corner, number in zip(corners, ids):
                (topLeft, topRight, bottomRight, bottomLeft) = corner[0].astype(int)
                # print(topLeft, topRight, bottomRight, bottomLeft)
                centerpoint = (int((topLeft[0] + bottomRight[0]) / 2), int((topLeft[1] + bottomRight[1]) / 2))
                if number == start_id:
                    start = centerpoint
                    # print("start:",start)
                    # print(topLeft, topRight, bottomRight, bottomLeft)
                    temp_globalmap[topLeft[0] - s:bottomRight[0] + s, topLeft[1] - s:bottomRight[1] + s] = 0
                    # print(temp_globalmap[bottomRight[0]-2:topLeft[0]+2,topLeft[1]-2:bottomRight[1]+2])

                elif number == goal_id:
                    goal = centerpoint
                    # print("goal:",goal)
                    # print(topLeft, topRight, bottomRight, bottomLeft)
                    temp_globalmap[topLeft[0] - s:bottomRight[0] + s, topLeft[1] - s:bottomRight[1] + s] = 0
                    # print(temp_globalmap[bottomRight[0]-2:topLeft[0]+2,topLeft[1]-2:bottomRight[1]+2])
                    self.goal_topLeft = topLeft
                    self.goal_topRight = topRight
                    pixel_to_distance = self.compute_pixel_to_distance()
                    # print("pixel_to_distance:",pixel_to_distance)

                else:
                    temp_globalmap[bottomRight[0] - s:topLeft[0] + s, topLeft[1] - s:bottomRight[1] + s] = 0
        else:
            pass

        globalmap = temp_globalmap.copy()

        if start is None:
            print("NOT detect start point")
        else:
            #globalmap[temp_globalmap == 2] = 0
            pass
        if goal is None:
            print("NOT detect goal point")
        else:
            #globalmap[temp_globalmap == 3] = 0
            pass

        # # obstacle_map:the map only have obstacle inside, no start and goal
        # self.obstacle_map = globalmap.copy()

        # obstacle inflation
        # 3x3 structuring element with connectivity 2
        struct = ndimage.generate_binary_structure(2, 2)
        # robot_size:radius of robot(unit:mm)
        margin = int(robot_size / pixel_to_distance)+1#margin at least need to be 1, can't be 0
        print(pixel_to_distance)
        #margin=1
        globalmap = ndimage.binary_dilation(globalmap, structure=struct, iterations=margin).astype(globalmap.dtype)

        # # add start and goal
        # globalmap[temp_globalmap == 2] = 2
        # globalmap[temp_globalmap == 3] = 3

        self.globalmap = globalmap
        self.start = start
        self.goal = goal

        return self.globalmap

    def compute_pixel_to_distance(self):
        # for get pixel_to_ditance value, use function "get_pixel_to_distance"
        # aruco marker:71mmx71mm
        # pixel_to_distance:return real world distance between two neighborhood pixels;unit:mm
        pixel_to_distance = 71 / math.dist(self.goal_topLeft, self.goal_topRight)  # unit:mm
        #print("Distance between two pixel is:", pixel_to_distance, "mm")

        self.pixel_to_distance = pixel_to_distance

        return self.pixel_to_distance

    def get_pixel_to_distance(self):
        return self.pixel_to_distance

    def get_globalmap(self):
        return self.globalmap

    def get_start(self):
        # get the center point of start point
        if self.start is None:
            print("NOT detect start point")
        else:
            pass
        return self.start

    def get_goal(self):
        # get the center point of goal point
        if self.goal is None:
            print("NOT detect goal point")
        else:
            pass
        return self.goal

    def compute_start(self, image, start_id=0):
        # Detect start by aruco marker
        # Return start position in (x,y) coordinate; if not detected return None
        # start_id: start aruco marker id

        # Define aruco marker detector
        arucoDict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_50)
        arucoParams = cv2.aruco.DetectorParameters_create()

        # Detect aruco marker
        (corners, ids, rejected) = cv2.aruco.detectMarkers(image, arucoDict, parameters=arucoParams)
        # print(corners, ids, rejected)
        start = None
        if ids is not None:
            for corner, number in zip(corners, ids):
                (topLeft, topRight, bottomRight, bottomLeft) = corner[0].astype(int)
                # print(topLeft, topRight, bottomRight, bottomLeft)
                centerpoint = (int((topLeft[0] + bottomRight[0]) / 2), int((topLeft[1] + bottomRight[1]) / 2))
                if number == start_id:
                    start = centerpoint
                    break
                else:
                    pass
        else:
            pass
        if start is None:
            print("NOT detect start point")
        else:
            print("start position is:", start)
        self.start = start

        return self.start

    def compute_goal(self, image, goal_id=1):
        # Detect goal by aruco marker
        # Return goal position in (x,y) coordinate; if not detected return None
        # goal_id=: goal aruco marker id

        # Define aruco marker detector
        arucoDict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_50)
        arucoParams = cv2.aruco.DetectorParameters_create()

        # Detect aruco marker
        (corners, ids, rejected) = cv2.aruco.detectMarkers(image, arucoDict, parameters=arucoParams)
        # print(corners, ids, rejected)
        goal = None
        if ids is not None:
            for corner, number in zip(corners, ids):
                (topLeft, topRight, bottomRight, bottomLeft) = corner[0].astype(int)
                # print(topLeft, topRight, bottomRight, bottomLeft)
                centerpoint = (int((topLeft[0] + bottomRight[0]) / 2), int((topLeft[1] + bottomRight[1]) / 2))
                if number == goal_id:
                    goal = centerpoint
                    self.goal_topLeft = topLeft
                    self.goal_topRight = topRight
                    break
                else:
                    pass
        else:
            pass
        if goal is None:
            print("NOT detect goal point")
        else:
            print("goal position is:", goal)
        self.goal = goal

        return self.goal

In [6]:
def compute_vertex(globalmap):
    # # get rid of start and goal area
    # globalmap[globalmap == 2] = 0
    # globalmap[globalmap == 3] = 0

    globalmap = np.uint8(np.dot(np.transpose(globalmap), 255))

    # apply canny edge detection
    edges = cv2.Canny(globalmap, 60, 160)

    # apply morphology close
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    morph = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)

    # get contours
    contours = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # get vertices
    vertices = []
    for i in range(len(contours[0])):
        peri = cv2.arcLength(contours[0][i], True)
        approx = cv2.approxPolyDP(contours[0][i], 0.01 * peri, True)
        vertice = []
        for i in range(len(approx)):
            vertice.append(approx[i][0].tolist())
        vertices.append(np.array(vertice))

    return vertices

In [7]:
def get_robot_position(image,robot_id=0):
    #Detect robot position by aruco marker
    #Return robot position; if not detected return None
    #robot_id: robot aruco marker id

    #Define aruco marker detector
    arucoDict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_50)
    arucoParams = cv2.aruco.DetectorParameters_create()

    robot_position = None
    angle=None

    #Detect Aruco marker
    (corners, ids, rejected) = cv2.aruco.detectMarkers(image, arucoDict,parameters=arucoParams)
    # print(corners, ids, rejected)
    if ids is not None:
        #print("aruco marker detected:",ids)
        for corner,number in zip(corners,ids):
            (topLeft, topRight, bottomRight, bottomLeft) = corner[0]
            centerpoint=(int((topLeft[0]+bottomRight[0])/2),int((topLeft[1]+bottomRight[1])/2))
            if number==robot_id:
                robot_position=centerpoint
                angle=angle_calculate(topLeft,topRight)
                break
            else:pass
    else:pass
    
    return robot_position,angle

def angle_calculate(pt1,pt2):  # function which returns angle between two points in the range of 0-359
    #angle_list = list(range(0,360,1))
    x=pt2[0]-pt1[0] 
    y=pt2[1]-pt1[1]
    angle=-np.arctan2(y,x) #takes 2 points nad give angle with respect to horizontal axis in range(-180,180)
    return angle

In [8]:
def kalman(current_state, pos_cam, motor_speed):
    global new_image
    #estimate x-y position with velocity
    #measurements and camera postion measurement
    f = KalmanFilter (dim_x=4, dim_z=4)

    # initial position
    f.x= np.transpose(current_state)
    print(current_state)
    
    #state transition matrix
   
    f.F = np.array([[1, 0, Ts, 0],
              [0, 1, 0, Ts], 
              [0, 0, 1, 0],
              [0, 0, 0, 1]])

    #measurement function H
    f.H = np.array([[1, 0, 0, 0],
                [0, 1, 0, 0],
                [0, 0, 1, 0],
                [0, 0, 0, 1]])

    #covariance matrix P
    f.P = np.array([[qp, 0, 0, 0], 
              [0, qp, 0, 0], 
              [0, 0, q_nu, 0], 
              [0, 0, 0, q_nu]])

    #measurement noise R (scalar)
    f.R = np.array([[rp, 0, 0, 0],
                [0, rp, 0, 0],
                [0, 0, r_nu, 0],
                [0, 0, 0, r_nu]])

    #process noise Q
    f.Q = Q_discrete_white_noise(dim=4, dt=0.1, var=0.13)
    
    if(new_image): #if image is up to date, take position from camera
        #print(motor_speed)
        #print(pos_cam)
        new_image=0#
        z = np.transpose(np.array([pos_cam[0],pos_cam[1], motor_speed[0],motor_speed[1]]))
    else:
    #pos=np.array([) #if image outdated, use state position instead of camera position
        z = np.array([current_state[0], current_state[1], motor_speed[0],motor_speed[1]])
    
    f.predict()
    f.update(z)
    print(f.x)
    #return state and covariance matrix
    return f.x

In [9]:
#global path

def global_path(start_pos, goal_pos, global_map):

    import pyvisgraph as vg
    import numpy as np
    import matplotlib.pyplot as plt

    start = np.array(start_pos)#np.array([0.5,0.5])
    goal = np.array(goal_pos)#np.array([5.25,6])
    print(start)
    print(goal)

    start_point = vg.Point(start[0], start[1])
    goal_point = vg.Point(goal[0], goal[1])

    vertices = np.array(global_map)#np.array([[[1,2],[3,2],[3,4],[1,4]],[[4,4],[6,4],[5,5]]])
   

    vertices = np.asarray(vertices)
    print(vertices)

    #plt.plot(vertices)
    #plt.show

    # Automate the creation of vg.Points for the obstacles

    max_vertices = 0

    for i in range(len(vertices)):
        if(len(vertices[i])>max_vertices):
            max_vertices = len(vertices[i])

    polygons = [[0 for i in range(max_vertices+1)] for j in range(len(vertices))]

    for i in range(len(vertices)):
        for j in range(len(vertices[i])):
            polygons[i][j] = vg.Point(vertices[i][j][0],vertices[i][j][1])
        polygons[i][(len(vertices[i])):(max_vertices+1)] = [polygons[i][0] for k in range(len(vertices[i]),max_vertices+1)]
        #polygons[i][range(len(vertices[i]),max_vertices)] = polygons[i][0]

    graph = vg.VisGraph()
    graph.build(polygons)
    #graph.buffer(margin,join_style=2)

    shortest_path = graph.shortest_path(start_point,goal_point)

    print(shortest_path)

    polygons_x = np.empty((len(polygons),len(polygons[1])+1))
    polygons_y = np.empty((len(polygons),len(polygons[1])+1))

    for i in range(len(polygons)):
        for j in range(len(polygons[i])):
            polygons_x[i][j] = polygons[i][j].x
            polygons_y[i][j] = polygons[i][j].y
        polygons_x[i][len(polygons[1])] = polygons_x[i][0]
        polygons_y[i][len(polygons[1])] = polygons_y[i][0]

    shortest_path_x = np.empty(len(shortest_path))
    shortest_path_y = np.empty(len(shortest_path))
    
    shortest = [[0 for i in range(2)] for j in range(len(shortest_path))]

    for i in range(len(shortest_path)):
        shortest_path_x[i] = shortest_path[i].x
        shortest_path_y[i] = shortest_path[i].y
        shortest[i][0] = shortest_path[i].x
        shortest[i][1] = shortest_path[i].y

    plt.plot(shortest_path_x,shortest_path_y)
    plt.plot(polygons_x.T,polygons_y.T)
    plt.show()
    return shortest
    
    


In [10]:
def global_nav(path):
    global state,index,direction
    
    #constantes
    v_max=400#mm/s
    #v_min=20
    Diameter=94#mm
    goal_margin=30#mm
    threshold=0.3
    
    #just to tune by test, names wrong
    Kp_d=0.8
    #CHECK GOAL
    if math.sqrt((robot_state[0]-path[index,0])**2+(robot_state[1]-path[index,1])**2)<goal_margin:
        if index+1>=np.size(path[:,0]):
            print('arrived')
            state=2
            return 0,0#works only in a function
        else:
            index=index+1
            print('index : ',index)
    
    #MOTORS
    obj_angle=np.arctan2(path[index,1]-robot_state[1],path[index,0]-robot_state[0])
    while(obj_angle-direction>np.pi):
        obj_angle=obj_angle-np.pi
    while(obj_angle-direction<=-np.pi):
        obj_angle=obj_angle+np.pi
    if abs(obj_angle-direction)>threshold:
        vitesse=0#-Kd_v*abs(obj_angle-direction[present])+Kp_v*abs(50-vitesse[present])#vitesse[present]
        print('droit')
        if vitesse<0:vitesse=0##v_min
    else:
        vitesse=100#-Ki_v*abs(50-vitesse[present])
    #derror=obj_dir(*position[present,:],*path[goal,:])+obj_dir(*position[present-1,:],*path[goal,:])-direction[present]-direction[present-1]
    speed_left=-(Kp_d*(obj_angle-direction))*Diameter#vitesse[present+1]    #0.2*(speed_l[present]+speed_l[present-1])/2    #-Kd_d*derror
    speed_right=(Kp_d*(obj_angle-direction))*Diameter#vitesse[present+1]    #0.2*(speed_r[present]+speed_r[present-1])/2+    #-Kd_d*derror
    #bounded speed for turn
    if speed_right<-v_max:speed_right=-v_max
    if speed_right>v_max:speed_right=v_max
    if speed_left<-v_max:speed_left=-v_max
    if speed_left>v_max:speed_left=v_max
    #bounded speed for turn+speedDC
    delta_speed=abs(speed_left-speed_right)
    if speed_left+vitesse>v_max:#PB si les deux au dessus
        speed_left=v_max
        speed_right=v_max-delta_speed
    elif speed_right+vitesse>v_max:
        speed_left=v_max-delta_speed
        speed_right=v_max
    else:
        speed_left=speed_left+vitesse
        speed_right=speed_right+vitesse
    direction=direction+(speed_right-speed_left)*0.1/(2*Diameter)
    return speed_left,speed_right

In [20]:
def navigation(path,obst):#vertex, goal_pos):
    global index,speed_left,speed_right,direction,state
    #state=1 # 0=gradient, 1=obstacle avoidance
    obstThrL = 1    
    obstThrH = 4     
    coef = 5 

    #goal_nb=0
    #speed_left=0
    #speed_right=0
    #direction=0


    #obst = np.array([0,0,0,0,0])  
    #await node.wait_for_variables({"prox.horizontal"})
    #obst = list(node.v.prox.horizontal)
    left_obst= (obst[0] + obst[1])//2
    right_obst=(obst[3] + obst[4])//2
    if state == 0:
        # if at least one prox sensor above treshold, go into obstacle avoidance (or condition)
        if (obst[0] > obstThrH):
            state = 1
        elif (obst[1] > obstThrH):
            state = 1
        elif (obst[3] > obstThrH):
            state = 1
        elif (obst[4] > obstThrH):
            state = 1
    elif state == 1:
      # if all prox sensors under treshold, go back to global nav (and condition)
        if obst[0] < obstThrL:
            if obst[1] < obstThrL : 
                if obst[3] < obstThrL : 
                    if obst[4] < obstThrL : 
                        state = 0
    #path=global_path([robot_state[0],robot_state[1]], goal_pos, vertex)           
    if  state  == 0:
        (speed_left, speed_right)=global_nav(path)#,goal_pos)
        motor_left_target = speed_left
        motor_right_target = speed_right
    else:
#        leds_top = [30,30,30]
        motor_left_target = speed_left - coef * (right_obst// 100)
        motor_right_target = speed_right - coef * (left_obst // 100)
        #path=global_path([robot_state[0],robot_state[1]], goal_pos, vertex)
        
    v = {
            "motor.left.target": [int(motor_left_target)],
            "motor.right.target": [int(motor_right_target)],
    }
    aw(node.set_variables(v))
    #await client.sleep(10*dt) # your long-running job goes here... here wait1s

In [21]:
from threading import Timer

class RepeatedTimer(object):
    def __init__(self, interval, function, *args, **kwargs):
        self._timer     = None
        self.interval   = interval
        self.function   = function
        self.args       = args
        self.kwargs     = kwargs
        self.is_running = False
        self.start()

    def _run(self):
        self.is_running = False
        self.start()
        self.function(*self.args, **self.kwargs)

    def start(self):
        if not self.is_running:
            self._timer = Timer(self.interval, self._run)
            self._timer.start()
            self.is_running = True

    def stop(self):
        self._timer.cancel()
        self.is_running = False




In [22]:
#acquire_data = False
#Ts = 0.1
thymio_data = []

def motors(left, right):
    return {
        "motor.left.target": [left],
        "motor.right.target": [right],
    }

def get_data():
    thymio_data.append({"prox":list(node["prox.horizontal"]),#not really useful??
                        "left_speed":node["motor.left.speed"],#for std
                        "right_speed":node["motor.right.speed"]})#for std
    #print("prox/mot")
    
def get_picture():
    #print("picture")
    #ret, frame = cap.read() #returns ret and the frame
    #pos_cam=get_robot_position(frame) #gets x-y measurements from camera and convert to mm
    #pos_cam[0]=pos_cam[0]*1.6
    #pos_cam[1]=pos_cam[1]*1.6
    new_image=1


In [19]:
global index,speed_left,speed_right,direction,state,pos_cam, new_image

v = {
        "motor.left.target": [0],
        "motor.right.target": [0],
    }
aw(node.set_variables(v))

direction=0
index=0
#cap = cv2.VideoCapture(1) #Resolution (640,480):camera provided in class
#ret, frame = cap.read() #returns ret and the frame
#globalmap=my_map.compute_globalmap(frame)
#start_pos=my_map.get_start()
robot_state[0]=0#start_pos[0]
robot_state[1]=0#start_pos[1]
#goal_pos= my_map.get_goal()
#vertex= compute_vertex(globalmap)
#drawmap=globalmap.copy()
#drawmap[drawmap == 2] = 0
#drawmap[drawmap == 3] = 0
#drawmap = np.float32(np.dot(np.transpose(drawmap),255))
#cv2.imshow("img",frame)
#cv2.imshow("draw",drawmap)
#cv2.waitKey(0)
new_image=0
start_pos=[robot_state[0], robot_state[1]]
#shortest_path= global_path(start_pos, goal_pos, global_map)

#launch timers (measure prox/motor;picture)
await node.wait_for_variables()
t1= RepeatedTimer(0.01, get_data)
t2= RepeatedTimer(1, get_picture)
#my_map=Map()
#print(vertex)
for i in range(100):#while state!=2: 
    await client.sleep(0.1)#timer_cam)#useful??
    ##await node.wait_for_variables({"motor.right.speed","motor.right.speed","prox.horizontal"})
    ##speed_left=node["motor.left.speed"]
    ##speed_right=node["motor.right.speed"]#get motor speed vx and vy converted to mm/s
    ##print("prox,motleft",list(node.v.prox.horizontal),node["motor.left.speed"])
    #await node.wait_for_variables({"prox.horizontal"})
    obst = list(node.v.prox.horizontal)
    print(obst)
    speed_left=node["motor.left.speed"]
    speed_right=node["motor.right.speed"]#get motor speed vx and vy converted to mm/s
    #prox = [x["prox"] for x in thymio_data]
    
    #speed_left = np.mean([x["left_speed"] for x in thymio_data])#  big list==>std for motors??
    #speed_right = np.mean([x["right_speed"] for x in thymio_data])
    #print("\nleft : ",speed_left,"\nright : ",speed_right)
    path=np.array([[0,1000],[0,-500],[657,254]])
    #vx=((speed_left+speed_right)/2)*np.cos(direction)
    #vy=((speed_left+speed_right)/2)*np.sin(direction)
    #motor_speed=np.array([vx,vy])
    #robot_state=kalman(robot_state,pos_cam, motor_speed)
    navigation(path,obst)#,vertex,goal_pos)
    robot_state=robot_state+[0.15*(speed_left+speed_right)/2*np.cos(direction),0.1*(speed_left+speed_right)/2*np.sin(direction),0,0]
    (robot_state[2],robot_state[3])=np.array([(speed_left+speed_right)/2*np.cos(direction),(speed_left+speed_right)/2*np.sin(direction)])
    
    #direction=direction+np.arctan2(robot_state[3],robot_state[2])
    print(direction,robot_state)
    
t1.stop()
t2.stop()
v = {
        "motor.left.target": [0],
        "motor.right.target": [0],
    }
aw(node.set_variables(v))

[0, 0, 0, 0, 0, 0, 0]
droit
0.12566370614359174 [0. 0. 0. 0.]
[0, 0, 0, 0, 0, 0, 0]
droit
0.24127431579569614 [0. 0. 0. 0.]
[0, 0, 0, 0, 0, 0, 0]
droit
0.3476360766756322 [0. 0. 0. 0.]
[0, 0, 0, 0, 0, 0, 0]
droit
0.44548889668517333 [0. 0. 0. 0.]
[0, 0, 0, 0, 0, 0, 0]
droit
0.5355134910939512 [0. 0. 0. 0.]
[0, 0, 0, 0, 0, 0, 0]
droit
0.6183361179500269 [0. 0. 0. 0.]
[0, 0, 0, 0, 0, 0, 0]
droit
0.6945329346576165 [0. 0. 0. 0.]
[0, 0, 0, 0, 0, 0, 0]
droit
0.7646340060285989 [0. 0. 0. 0.]
[0, 0, 0, 0, 0, 0, 0]
droit
0.8291269916899027 [0. 0. 0. 0.]
[0, 0, 0, 0, 0, 0, 0]
droit
0.8884605384983022 [0. 0. 0. 0.]
[0, 0, 0, 0, 0, 0, 0]
droit
0.9430474015620297 [0. 0. 0. 0.]
[0, 0, 0, 0, 0, 0, 0]
droit
0.9932673155806591 [0. 0. 0. 0.]
[0, 0, 0, 0, 0, 0, 0]
droit
1.039469636477798 [0. 0. 0. 0.]
[0, 0, 0, 0, 0, 0, 0]
droit
1.081975771703166 [0. 0. 0. 0.]
[0, 0, 0, 0, 0, 0, 0]
droit
1.1210814161105045 [0. 0. 0. 0.]
[0, 0, 0, 0, 0, 0, 0]
droit
1.157058608965256 [0. 0. 0. 0.]
[0, 0, 0, 0, 0, 0, 0]
dr

[0, 0, 0, 0, 0, 0, 0]
1.624116909764507 [ 20.37887823 615.14664784  -5.32953207  99.85787945]
[0, 0, 0, 0, 0, 0, 0]
1.6240834961045145 [ 19.57994892 625.13245358  -5.32619545  99.85805747]
[0, 0, 0, 0, 0, 0, 0]
1.6239952605537635 [ 18.78234126 635.11830629  -5.3173844   99.85852704]
[0, 0, 0, 0, 0, 0, 0]
1.6238537267641724 [ 17.98685361 645.10423415  -5.30325099  99.85927863]
[0, 0, 0, 0, 0, 0, 0]
1.6236602344411146 [ 17.19426427 655.09026444  -5.28392889  99.8603029 ]
[0, 0, 0, 0, 0, 0, 0]
1.6234159422365704 [ 16.40533423 665.07642352  -5.25953364  99.86159074]
[0, 0, 0, 0, 0, 0, 0]
1.6231218292950194 [ 15.6208098  675.06273685  -5.23016283  99.86313332]


In [15]:
v = {
        "motor.left.target": [0],
        "motor.right.target": [0],
    }
aw(node.set_variables(v))